In [1]:
#Data from yahoo finance package
import yfinance as yf
import pandas as pd 
import numpy as np
import os

import warnings
warnings.filterwarnings("ignore")

In [2]:
path = "E:/NUS_Exchange/Research/Datasets/"
try:
    os.mkdir(path + "Output")
    os.mkdir(path + "Data")
    os.mkdir(path + "Output/GridSearchCV_Result")
    os.mkdir(path + "Output/HoldOutValidation")
except:
    print("Folder already exists")

Folder already exists


In [3]:
def readX(pathname):
    '''
    Read X related file
    '''
    df = pd.read_csv(pathname, index_col=[0], header=[0])
    return df

Some functions calculations are inspired by the website:
https://blog.quantinsti.com/build-technical-indicators-in-python/

In [4]:
# EMA calculation
def EWMA(data): 
    EMA = pd.Series(data['Close'].ewm(span = 15, min_periods = 15 - 1).mean(), 
                 name = '15_Days_EWMA') 
    data = data.join(EMA) 
    return data

# Returns RSI values
def RSI(close, periods = 15):
    
    close_delta = close.diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()

    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi

In [5]:
# MFI index definition
def gain(x):
    return ((x > 0) * x).sum()

def loss(x):
    return ((x < 0) * x).sum()

# Calculate money flow index
def MFI(high, low, close, volume, n=15):
    typical_price = (high + low + close)/3
    money_flow = typical_price * volume
    mf_sign = np.where(typical_price > typical_price.shift(1), 1, -1)
    signed_mf = money_flow * mf_sign
    mf_avg_gain = signed_mf.rolling(n).apply(gain, raw=True)
    mf_avg_loss = signed_mf.rolling(n).apply(loss, raw=True)
    return (100 - (100 / (1 + (mf_avg_gain / abs(mf_avg_loss))))).to_numpy()

In [6]:
# ATR index function
def ATR(high, low, close, n=15):
    tr = np.amax(np.vstack(((high - low).to_numpy(), (abs(high - close)).to_numpy(), (abs(low - close)).to_numpy())).T, axis=1)
    return pd.Series(tr).rolling(n).mean().to_numpy()

In [7]:
# ForceIndex function
def ForceIndex(data, ndays = 10): 
    FI = pd.Series(data['Close'].diff(ndays) * data['Volume'], name = 'ForceIndex') 
    data = data.join(FI) 
    return data

In [8]:
# MACD
def MACD(data, slowspan = 26, quickspan = 12):
    slow = data['Close'].transform(lambda x: x.ewm(span=slowspan, adjust=False).mean())
    quick = data['Close'].transform(lambda x: x.ewm(span=quickspan, adjust=False).mean())
    return slow - quick

# 1. Price Data indicator generating

In [9]:
# Recursively put .csv files into list
import os
 
def list_dir(file_dir):
    # list_csv = []
    dir_list = os.listdir(file_dir)
    for cur_file in dir_list:
        path = os.path.join(file_dir,cur_file)
        if os.path.isfile(path):
            # print("{0} : is file!".format(cur_file))
            dir_files = os.path.join(file_dir, cur_file)
        if os.path.splitext(path)[1] == '.csv':
            csv_file = os.path.join(file_dir, cur_file)
            # print(os.path.join(file_dir, cur_file))
            # print(csv_file)
            list_csv.append(csv_file)
        if os.path.isdir(path):
            # print("{0} : is dir".format(cur_file))
            # print(os.path.join(file_dir, cur_file))
            list_dir(path)
    return list_csv
 
 
if __name__ == '__main__':
    paths = r'E:/NUS_Exchange/Research/Datasets/Data/Price'
    list_csv = []
    list_dir(file_dir=paths)
    print(list_csv)

['E:/NUS_Exchange/Research/Datasets/Data/Price\\XDI_AX.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price\\XEC_AX.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price\\XJR_AX.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price\\XLD_AX.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price\\XNT_AX.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price\\XNV_AX.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price\\XVI_AX.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price\\^AFLI.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price\\^AORD.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price\\^ATLI.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price\\^ATOI.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price\\^AXAF.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price\\^AXAT.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price\\^AXBW.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price\\^AXDJ.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price\\^AXEJ.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price\\^

In [ ]:
# Trials
# companiesName.iloc[0][0]
# x = yf.Ticker("1AL.F")
# x_historical = x.history(start="2020-01-01", end="2022-12-31")
# x_historical.to_csv(path+"/Data/StockPriceAndDividend/ConsDisFundamentalData/" + "1AL.F" + ".csv")

As we assume 30 days to be a window to train the datasets, then we will create the relavant indicators from price data

In [10]:
def pricedataobtain(filename):
    x_historical = readX(filename)
    x_df = x_historical.copy()
    # All of them are yesterday data
    x_df["OpenClose_spread"] = x_df.apply(lambda x: x["Close"] - x["Open"], axis=1).shift(1).fillna(0)
    x_df["Highlow_spread"] = x_df.apply(lambda x: x["High"] - x["Low"], axis=1).shift(1).fillna(0)
    # All of them are Moving average data using close price
    # Stock price data
    x_df["5_Days_MA"] = x_df[["Close"]].rolling(window=5).mean()
    x_df["10_Days_MA"] = x_df[["Close"]].rolling(window=10).mean()
    x_df["15_Days_MA"] = x_df[["Close"]].rolling(window=15).mean()
    x_df["30_Days_MA"] = x_df[["Close"]].rolling(window=30).mean()
    x_df["5_Days_VAR"] = x_df[["Close"]].rolling(window=5).var()
    x_df["15_Days_VAR"] = x_df[["Close"]].rolling(window=15).var()
    x_df["30_Days_VAR"] = x_df[["Close"]].rolling(window=30).var()

    # Technical indicators
    EWMA(x_df)


    # Macro economic indicators

    # Drop meaningless features
    x_df = x_df.drop(columns=["Open", "High", "Low", "Volume", "Adj_Close"], axis=1)
    x_df = x_df[16:]
    x_df = x_df.rename(columns={"Close" : "Price"})

    # Transform the index to date time
    x_df['Date'] = pd.to_datetime(x_df.index)
    x_df = x_df.set_index(['Date'], drop=True)

    x_df.to_csv(path+"/Data/Price_Index/" + filename[45:] + ".csv")

for i in list_csv:
    pricedataobtain(i)

TypeError: cannot do positional indexing on DatetimeIndex with these indexers [2017-01-01 00:00:00] of type Timestamp

# 2. Fundamental Data cleaning

In [4]:
def list_dir(file_dir):
    # list_csv = []
    dir_list = os.listdir(file_dir)
    for cur_file in dir_list:
        path = os.path.join(file_dir,cur_file)
        if os.path.isfile(path):
            # print("{0} : is file!".format(cur_file))
            dir_files = os.path.join(file_dir, cur_file)
        if os.path.splitext(path)[1] == '.csv':
            csv_file = os.path.join(file_dir, cur_file)
            # print(os.path.join(file_dir, cur_file))
            # print(csv_file)
            list_csv.append(csv_file)
        if os.path.isdir(path):
            # print("{0} : is dir".format(cur_file))
            # print(os.path.join(file_dir, cur_file))
            list_dir(path)
    return list_csv
 
 
if __name__ == '__main__':
    paths = r'E:/NUS_Exchange/Research/Datasets/Data/Fundamental'
    list_csv = []
    list_dir(file_dir=paths)
    print(list_csv)

['E:/NUS_Exchange/Research/Datasets/Data/Fundamental\\ConsDisFundamentalData.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Fundamental\\ConsStapFundamentalData.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Fundamental\\EnergyFundamentalData.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Fundamental\\FinanceFundamentalData.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Fundamental\\HealthcareFundamentalData.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Fundamental\\IndustrialFundamentalData.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Fundamental\\ITFundamentalData.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Fundamental\\LargeCapFundamentalData.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Fundamental\\MaterialFundamentalData.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Fundamental\\MediumCapFundamentalData.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Fundamental\\SmallCapFundamentalData.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Fundamental\\TeleFundamentalData.csv', 'E:/NUS_Exch

In [ ]:
# These are the fundamental data
'''
ConsDisFundamentalData_df = readX(path+"/Fundamental/ConsDisFundamentalData.csv")
ConsStapFundamentalData_df = readX(path+"/Fundamental/ConsStapFundamentalData.csv")
EnergyFundamentalData_df = readX(path+"/Fundamental/EnergyFundamentalData.csv")
FinanceFundamentalData_df = readX(path+"/Fundamental/FinanceFundamentalData.csv")
HealthcareFundamentalData_df = readX(path+"/Fundamental/HealthcareFundamentalData.csv")
IndustrialFundamentalData_df = readX(path+"/Fundamental/IndustrialFundamentalData.csv")
ITFundamentalData_df = readX(path+"/Fundamental/ITFundamentalData.csv")
MaterialFundamentalData_df = readX(path+"/Fundamental/MaterialFundamentalData.csv")
MediumCapFundamentalData_df = readX(path+"/Fundamental/MediumCapFundamentalData.csv")
SmallCapFundamentalData_df = readX(path+"/Fundamental/SmallCapFundamentalData.csv")
TeleFundamentalData_df = readX(path+"/Fundamental/MediumCapFundamentalData.csv")
UtilitiesFundamentalData_df = readX(path+"/Fundamental/SmallCapFundamentalData.csv")
'''

In [5]:
# Deal with nan data
# Also, store the name of the stocks into one csv files
Name = pd.DataFrame()
for i in list_csv:
    x_df = readX(i)
    x_df = x_df.dropna(how="any")
    x_df.to_csv(path+"/Data/Fundamental_Stock/" + i[50:])
    Name.append(pd.DataFrame(x_df.index))
Name = Name.drop_duplicates()
Name.index=pd.Series(range(0,2089))
Name.to_csv(path+"/Data/Fundamental_Stock/Name.csv")

In [7]:
if __name__ == '__main__':
    paths = r'E:/NUS_Exchange/Research/Datasets/Data/Fundamental_Stock'
    list_csv = []
    list_dir(file_dir=paths)
    print(list_csv)

['E:/NUS_Exchange/Research/Datasets/Data/Fundamental_Stock\\ConsDisFundamentalData.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Fundamental_Stock\\ConsStapFundamentalData.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Fundamental_Stock\\EnergyFundamentalData.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Fundamental_Stock\\FinanceFundamentalData.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Fundamental_Stock\\HealthcareFundamentalData.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Fundamental_Stock\\IndustrialFundamentalData.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Fundamental_Stock\\ITFundamentalData.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Fundamental_Stock\\LargeCapFundamentalData.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Fundamental_Stock\\MaterialFundamentalData.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Fundamental_Stock\\MediumCapFundamentalData.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Fundamental_Stock\\SmallCapFundamentalData.csv', 'E:/NUS_Exchange/Research

In [9]:
# Also, store the name of the stocks into one csv files
Name = pd.DataFrame()
for i in list_csv:
    x_df = readX(i)
    Name = Name.append(pd.DataFrame(x_df.index))
Name = Name.drop_duplicates()
Name.index=pd.Series(range(0,900))
Name.to_csv(path+"/Data/Fundamental_Stock/Name.csv")

# After the data cleaning, only 900 stocks are stored in the name file

# 3. Download stock prices included in the S&P/ASX 50 from yahoo finance

In [11]:
# Trials
# companiesName.iloc[0][0]

stockcode = "XRO.AX"

x = yf.Ticker(stockcode)
x_historical = x.history(start="2015-01-01", end="2022-12-31")
x_historical.to_csv(path+"/Data/Price_Stock/" + stockcode + ".csv")

I have ignored some stocks, such as TLC, WTC, EDV, COL

In [12]:
if __name__ == '__main__':
    paths = r'E:/NUS_Exchange/Research/Datasets/Data/Price_Stock'
    list_csv = []
    list_dir(file_dir=paths)
    print(list_csv)

['E:/NUS_Exchange/Research/Datasets/Data/Price_Stock\\AIA.AX.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price_Stock\\ALL.AX.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price_Stock\\AMC.AX.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price_Stock\\ANZ.AX.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price_Stock\\APA.AX.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price_Stock\\ASX.AX.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price_Stock\\BHP.AX.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price_Stock\\BXB.AX.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price_Stock\\CBA.AX.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price_Stock\\COH.AX.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price_Stock\\CPU.AX.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price_Stock\\CSL.AX.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price_Stock\\FMG.AX.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price_Stock\\FPH.AX.csv', 'E:/NUS_Exchange/Research/Datasets/Data/Price_Stock\\GMG.AX.csv', 'E:/NUS_E

As we assume 30 days to be a window to train the datasets, then we will create the relavant indicators from price data

In [14]:
def pricedataobtain(filename):
    x_historical = readX(filename)
    x_df = x_historical.copy()
    # All of them are yesterday data

    # Technical fundamental indicators:
    x_df["OpenClose_spread"] = x_df.apply(lambda x: x["Close"] - x["Open"], axis=1).shift(1).fillna(0)
    x_df["Highlow_spread"] = x_df.apply(lambda x: x["High"] - x["Low"], axis=1).shift(1).fillna(0)
    # All of them are Moving average data using close price
    x_df["5_Days_MA"] = x_df[["Close"]].rolling(window=5).mean()
    x_df["10_Days_MA"] = x_df[["Close"]].rolling(window=10).mean()
    x_df["15_Days_MA"] = x_df[["Close"]].rolling(window=15).mean()
    x_df["30_Days_MA"] = x_df[["Close"]].rolling(window=30).mean()
    x_df["5_Days_VAR"] = x_df[["Close"]].rolling(window=5).var()
    x_df["15_Days_VAR"] = x_df[["Close"]].rolling(window=15).var()
    x_df["30_Days_VAR"] = x_df[["Close"]].rolling(window=30).var()
    
    # Technical advanced indicators:
    x_df = EWMA(x_df)
    x_df["15_Days_RSI"] = RSI(x_df['Close'])
    x_df["15_Days_MFI"] = MFI(x_df["High"], x_df["Low"], x_df["Close"], x_df["Volume"])
    x_df["15_Days_ATR"] = ATR(x_df["High"], x_df["Low"], x_df["Close"])    
    x_df = ForceIndex(x_df)
    x_df["Typical_MACD"] = MACD(x_df)


    # Dropout meaningless features
    x_df = x_df.drop(columns=["Open", "High", "Low"], axis=1)
    x_df = x_df[31:]
    x_df = x_df.rename(columns={"Close" : "Price"})

    # Transform the index to date time
    x_df['Date'] = pd.to_datetime(x_df.index)
    x_df['Date'] = x_df['Date'].apply(lambda x: x.replace(tzinfo=None))
    x_df = x_df.set_index(['Date'], drop=True)

    x_df.to_csv(path+"/Data/ASX_stockconsidering/" + filename[51:] + ".csv")

for i in list_csv:
    pricedataobtain(i)